<hr style="height:5px;border-width:0;color:orange;background-color:green">
<center> <span style="font-size: 32px;"> Bob: Recommendation Module </span>

<a id="0"></a>
<font color=green size=+3><b>0 - Module Summary</b></font>

## Machine Learning approach to contrast the manual selection and recommendation problem
model en verda, primero manual y despues contraste con modelo
state limitations (data) and future improvements (more tailored model w/ collaborative filtering to see which of them go together)

The objective of this model will be to assess the probability of the group of selected people to engage or not by purchasing an em account.

For that I will use the following data:
- To be consistent, only customers that have been considered active this last month will be considered.
- TARGET will be em account.
- Distribution of products that a given customer has in last partition (except em account)
- revenue computed in last manual step
- months being active

This model is extremely primitive and thus very limited. 
The final goal is to predict the probability of a client that has no products, purchasing an em account. In order to do so it will only have access to a list of products == 0 and the number of months which that client has been active in the app and 3 values of revenue (0,10,20) but mostly 0.
In order to reach this goal the model will have to learn patterns from clients' products list (containing ones), number of months (most consistent feature across the dataset) and values of revenue (greater than zero) which will lead to not very reliable results.

About looking for similar profiles in other months, again the limitation is the features we are inputting. If anything the only thing that will enrich the model are demographics:
- Age range for example
- Entry channel..
- Gender at some point...

<a id="0.2"></a>
<font color=green size=+2><b>0.2 - Import Libraries</b></font>

<a id="0.3"></a>
<font color=green size=+2><b>0.3 - User Functions</b></font>

<a id="1"></a>
<font color=green size=+3><b>1 - Data Understanding</b></font>

### **Features**

In [1]:
import pandas as pd
import numpy as np

## VISUALIZATION
import matplotlib.pyplot as plt
import seaborn as sns

## SKLEARN
import sklearn
from sklearn import metrics
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn import model_selection
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
import xgboost as xgb

# FORMAT
pd.options.display.float_format = '{:,.2f}'.format

c:\Users\javce\anaconda3\envs\nuclio_env\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
df = pd.read_csv('data/Bobs_df_delivery.csv', index_col=0)
df.head()

,pk_partition,country_id,region_code,gender,age,salary,short_term_deposit,loans,mortgage,funds,...,entry_date,entry_channel,active_customer,Total_products,year_partition,month_partition,year_month_partition,quarter_entry,year_month_entry,salary_LN
pk_cid,,,,,,,,,,,,,,,,,,,,,
1375586,2018-01-28,ES,Malaga,H,36,"87,218.10",0,0,0,0,...,2018-01-12,others,1,1,2018,1,2018-01,2018-Q1,2018-01,11.38
1050611,2018-01-28,ES,Ciudad Real,V,24,"35,548.74",0,0,0,0,...,2015-08-10,KHE,0,1,2018,1,2018-01,2015-Q3,2015-08,10.48
1050612,2018-01-28,ES,Ciudad Real,V,23,"122,179.11",0,0,0,0,...,2015-08-10,KHE,0,1,2018,1,2018-01,2015-Q3,2015-08,11.71
1050613,2018-01-28,ES,Zaragoza,H,23,"119,775.54",1,0,0,0,...,2015-08-10,KHD,0,1,2018,1,2018-01,2015-Q3,2015-08,11.69
1050614,2018-01-28,ES,Zaragoza,V,24,"89,257.08",0,0,0,0,...,2015-08-10,KHE,1,1,2018,1,2018-01,2015-Q3,2015-08,11.40


In [3]:
df.reset_index(inplace=True)

In [36]:
# Explain the approach of the model

## Best client

# longest period being active
# number of products
# revenue of those products
#==============================
# VARIABLES

# clients_id: distinct customer ids
# active_id: distinct customer ids active last month
# non_active_id: distinct customer ids non active last month
# active: df with all partitions of customers in active_id
# non_active: df with all partitions of customers in non_active_id
# rank_active_df: rank active customers in terms of revenue

As it was shown during the analysis phase, there is a problem with the engagement of the current client base that can be seen in the amount of average amount of products per client on a given month. Right now this value is below one, meaning that we have more customers without products than with products while at the very least this ratio should be at one.

This found issue aligns with the overall strategy of the company right now, which is to reorient the commercial focus towards increasing the penetration of the current product portfolio in order to increase profits from the current customer base rather than doing any other campaign for acquiring new customers.

Having this in mind, in order to recommend products to the current clientbase to maximize profitability out of it, the focus will be placed on customers that have no products. An attempt will be made at manually characterize these customers and understand which products to offer them, though we already have relevant information that we can carry from the Analysis phase. Once this manual model has been put in place, it can be constrasted with results from a machine learning model that would yield probabilities of purchase for this manually chosen customers.

Active / non-active: even though problems, if we believe how active is calculated there should be more probabilities of an active customer to buy a products than one non acitve. In any case if the way active is computed in the application is modified this model could be retrained and reused.

The model described above is very basic... etc and is only aimed at.
Future iterations of it can focus on...

Features I am using and why

Assumption that if the customer has products but appears as non active, we still make revenue out of it.

Explain step by step process>
1. manual model
2. Limitations
3. Machine Leaarning in the future??

# **Manual Selection of 10k customers**

In [4]:
products = ['short_term_deposit', 'loans', 'mortgage', 'funds','securities', 'long_term_deposit', 'credit_card', 'payroll','pension_plan', 'payroll_account', 'emc_account', 'debit_card','em_acount']

In [14]:
# # First off, let's start by selecting only customers present in last month

tgt_id = df.loc[df['pk_partition']=='2019-05-28','pk_cid'].unique().tolist()
len(tgt_id)

440248

In [15]:
len(set(tgt_id))

440248

In [16]:
# keep only meaningful features
tgt_df = pd.concat([df[['pk_partition','pk_cid']],df[products],df['active_customer']],axis=1)
# filter out clients not in tgt_id
tgt_df = tgt_df[tgt_df['pk_cid'].isin(tgt_id)]
# # remove last month because it will used as a target in the modelling phase
# tgt_df = tgt_df[tgt_df.pk_partition != '2019-05-28']

In [17]:
len(tgt_df['pk_cid'].unique())

440248

In [18]:
tgt_df.columns

Index(['pk_partition', 'pk_cid', 'short_term_deposit', 'loans', 'mortgage',
       'funds', 'securities', 'long_term_deposit', 'credit_card', 'payroll',
       'pension_plan', 'payroll_account', 'emc_account', 'debit_card',
       'em_acount', 'active_customer'],
      dtype='object')

The list of revenues as reported by Erin:
- €10 for each account
- €40 for savings and investment products (plans, funds,...)
- €60 for financing products (loans and cards)

In [19]:
# Taking into account the list above + the current columns of the active df:
revenues_list = [1,1,40,60,60,40,40,40,60,10,40,0,10,60,10,1]

In [20]:
tgt_df = tgt_df*revenues_list

In [21]:
tgt_df.loc[:,'sum'] = tgt_df[products].sum(axis=1)
tgt_df.head()

,pk_partition,pk_cid,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,credit_card,payroll,pension_plan,payroll_account,emc_account,debit_card,em_acount,active_customer,sum
0,2018-01-28,1375586,0,0,0,0,0,0,0,0,0,0,0,0,10,1,10
1,2018-01-28,1050611,0,0,0,0,0,0,0,0,0,0,0,0,10,0,10
2,2018-01-28,1050612,0,0,0,0,0,0,0,0,0,0,0,0,10,0,10
3,2018-01-28,1050613,40,0,0,0,0,0,0,0,0,0,0,0,0,0,40
4,2018-01-28,1050614,0,0,0,0,0,0,0,0,0,0,0,0,10,1,10


In [22]:
tgt_df.isnull().sum()

pk_partition          0
pk_cid                0
short_term_deposit    0
loans                 0
mortgage              0
funds                 0
securities            0
long_term_deposit     0
credit_card           0
payroll               0
pension_plan          0
payroll_account       0
emc_account           0
debit_card            0
em_acount             0
active_customer       0
sum                   0
dtype: int64

In [23]:
rank_tgt = pd.DataFrame(index=tgt_id, columns=['revenue','months'])

# Revenue: total revenue over time from each customer: product x revenue @ each partition
# Months: number of months the client has been active in the app.
# Reminder - This selection of customers is based on the fact that they have been active in the app last month.

In [24]:
rank_tgt = tgt_df.groupby('pk_cid').agg({'sum': 'sum', 'active_customer': 'sum'})
rank_tgt.columns = ['revenue', 'months']
rank_tgt.to_csv('rank_tgt.csv')
rank_tgt.head()

,revenue,months
pk_cid,,
16063,0,5
16203,50,5
16502,130,8
17457,1110,17
17590,0,11


In [26]:
rank_tgt.shape

(440248, 2)

In [ ]:
# rank_tgt = pd.read_csv('rank_tgt.csv',index_col=0)

In [27]:
rank_tgt.sort_values(by='revenue', ascending=False,inplace=True)
rank_tgt

,revenue,months
pk_cid,,
1128353,4880,17
1116675,4810,17
1136671,4670,17
1070525,4440,17
1133500,4420,17
...,...,...
1330066,0,0
1330062,0,0
1330060,0,0


In [60]:
rank_tgt.describe()

,revenue,months
count,"437,479.00","437,479.00"
mean,230.92,5.32
std,373.28,7.06
min,0.00,0.00
25%,50.00,0.00
50%,170.00,0.00
75%,170.00,12.00
max,"4,880.00",17.00


**OLD COMMENTS, MODIFY BASED ON NEW INFORMATION**

mean revenue is higher than the median revenue, which means that the top percentage of clients yield a lot of money while there are many that mean 0€ even though they are active

for the months is the opposte, the median is 17 months, meaning half of the customers stay active for 17 months, while the mean stay is 13 months

In [28]:
def get_products(row):
    result = []
    index = row['pk_cid']
    for p in products:
        if row[p]==1:
            result.append(p)
    return [result,index]

In [29]:
# number of products each customer has during last month
products_per_customer = df.loc[df['pk_partition']=='2019-05-28'].apply(get_products, axis=1)    
products_per_customer = products_per_customer.apply(pd.Series)
products_per_customer.columns = ['products', 'id']
products_per_customer.head()

,products,id
5485815,[em_acount],657826
5485816,[],657817
5485817,"[credit_card, payroll, pension_plan, payroll_a...",657986
5485818,"[long_term_deposit, em_acount]",657905
5485819,[em_acount],657336


In [30]:
products_per_customer.shape

(440248, 2)

In [31]:
products_per_customer.set_index('id', inplace=True)

In [33]:
rank_tgt = pd.merge(rank_tgt, products_per_customer, how='left', left_index=True, right_index=True)
rank_tgt['products'].value_counts().head(10)

[em_acount]                                             255094
[]                                                      108751
[debit_card, em_acount]                                  22400
[emc_account, em_acount]                                  7825
[payroll, pension_plan, payroll_account, debit_card]      6009
[emc_account]                                             5745
[payroll_account]                                         4704
[payroll, pension_plan, payroll_account]                  4335
[payroll_account, debit_card]                             3109
[emc_account, debit_card, em_acount]                      2279
Name: products, dtype: int64

In [34]:
rank_tgt['products_str'] = rank_tgt['products'].astype('str')
rank_tgt['products_str'] = rank_tgt['products_str'].str.strip('[]').str.replace("'","")
len(rank_tgt[rank_tgt['products_str']==''])

108751

In [35]:
# 108751 customers appearing in the last month w/o products
# from here we will have to make a selection of 10k

# First step:
# which ones of them have been active last month??

index_group1 = df.loc[(df['pk_partition']=='2019-05-28')&(df['active_customer']==1)&(df['Total_products']==0), 'pk_cid'].unique()
len(index_group1)

2769

In [38]:
# remove these customers from the ranking and add them to the 10k list
df_10k = rank_tgt[rank_tgt.index.isin(index_group1)]
rank_tgt = rank_tgt[~rank_tgt.index.isin(index_group1)]
print(len(rank_tgt))
print(len(df_10k))
print(len(rank_tgt) + len(df_10k))

437479
2769
440248


In [39]:
df_10k

,revenue,months,products,products_str
pk_cid,,,,
1045535,2430,16,[],
1116106,2260,17,[],
1020461,2260,17,[],
1119050,2030,17,[],
1209899,1950,17,[],
...,...,...,...,...
1312708,0,5,[],
1314286,0,11,[],
1313313,0,17,[],


In [ ]:
# now for the remaining 7231 customers:
# from the rank df, the ones that have no products, those order by revenue first and months as a second factor

In [45]:
rank_tgt.loc[(rank_tgt['products_str']=='')].sort_values(by=['revenue','months'], ascending=False)

,revenue,months,products,products_str
pk_cid,,,,
1250924,2030,16,[],
1126774,1880,15,[],
1195576,1840,14,[],
1320970,1820,16,[],
1016939,1740,10,[],
...,...,...,...,...
1330066,0,0,[],
1330062,0,0,[],
1330060,0,0,[],


In [ ]:
# we can see that there are customers that even though have not been active last month and have no products currently, have been active in the past and have had products
# The assumption is that these customers would be easier to reengage that customers that have had less relation with the company in the past
# so we will pick the first 7343

In [55]:
group2 = rank_tgt.loc[(rank_tgt['products_str']=='')].sort_values(by=['revenue','months'], ascending=False).iloc[:7231]
index_group2 = group2.index.tolist()
rank_tgt = rank_tgt[~rank_tgt.index.isin(index_group2)]

In [57]:
df_10k = pd.concat([df_10k,group2])

In [59]:
df_10k.drop(['products','products_str'], axis=1, inplace=True)
df_10k

,revenue,months
pk_cid,,
1045535,2430,16
1116106,2260,17
1020461,2260,17
1119050,2030,17
1209899,1950,17
...,...,...
1119048,50,0
1352778,50,0
1272462,50,0


# **Which product do we offer to them**

During the analysis phase, it has been discussed why the main product of the company (EM account), is the most solid product of the company: the one that attracts more new customers and the one with better engagement. As a quick check, let's see which are the most popular product(s) this last month in the list of customers that we didn't select as the 10k.

In [69]:
rank_tgt['products'].value_counts().head(10)

[em_acount]                                             255094
[]                                                       98751
[debit_card, em_acount]                                  22400
[emc_account, em_acount]                                  7825
[payroll, pension_plan, payroll_account, debit_card]      6009
[emc_account]                                             5745
[payroll_account]                                         4704
[payroll, pension_plan, payroll_account]                  4335
[payroll_account, debit_card]                             3109
[emc_account, debit_card, em_acount]                      2279
Name: products, dtype: int64

As seen above, most customers have only EM account, then we still have a great deal of customers that have no products. Interestingly, debit card and emc account seem to be the products of choice when a customer decides to purchase a second product, specially the former.

# **Now we know the product, let's see what are the chances of them buying it**

In order to estimate estimate the probabilities of them actually purchasing an EM account, let's start with a very primitive and simple model.

This is the follow up of the cheapest and simplest of the manual models and it is focused on engaging customers. 

Limitations

Scope: supervised to predict last month em account 1 based on revenue, month and age\
Methodology:\
transform dataframe so it can be fed in model\
train\
test\
validate\
expose to 10k

In [ ]:
## start prepartion of the dataset for machine learning model
# drop features 
# one hot encode etc

In [ ]:
# the prediction target will be whether the customers have EM account or not the last month.
# to train the model, we can use all the available partitions, as we have one full "season" so we have many
# customers in the database that have the EM account, some of them are new others have been more time
# we will have all sort of revenue values and months active in the app
# in order to add a bit of flavor, because people w/ 0 revenue will be very difficult to asses
# let's add the age, which seems one of the parameters that plays a role in the type of products the different people would buy


Add justifications for the following:
- preparation of the df, feature selection and trnaformation
- choosing xgboost
- Results, metrics, which ones am I looking at
- numbers of FP,...
- training, testing, validation,.. similarities in data split and thus such good results. Effect of old/recent data, not ncecesarily.
- Predict proba not being the actual probas? article in medium shared by David or Thorsten.

In [71]:
# 1. Recompute revenue and months w/o taking into account last partition

In [73]:
# keep only meaningful features
tgt_model_df = pd.concat([df[['pk_partition','pk_cid']],df[products],df['active_customer']],axis=1)
# filter out clients not in tgt_id
tgt_model_df = tgt_model_df[tgt_model_df['pk_cid'].isin(tgt_id)]
# remove last month because it will used as a target in the modelling phase
tgt_model_df = tgt_model_df[tgt_model_df.pk_partition != '2019-05-28']

In [74]:
tgt_model_df = tgt_model_df*revenues_list
tgt_model_df.loc[:,'sum'] = tgt_model_df[products].sum(axis=1)
tgt_model_df.head()

,pk_partition,pk_cid,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,credit_card,payroll,pension_plan,payroll_account,emc_account,debit_card,em_acount,active_customer,sum
0,2018-01-28,1375586,0,0,0,0,0,0,0,0,0,0,0,0,10,1,10
1,2018-01-28,1050611,0,0,0,0,0,0,0,0,0,0,0,0,10,0,10
2,2018-01-28,1050612,0,0,0,0,0,0,0,0,0,0,0,0,10,0,10
3,2018-01-28,1050613,40,0,0,0,0,0,0,0,0,0,0,0,0,0,40
4,2018-01-28,1050614,0,0,0,0,0,0,0,0,0,0,0,0,10,1,10


In [100]:
rank_model_df = tgt_model_df.groupby('pk_cid').agg({'sum': 'sum', 'active_customer': 'sum'})
rank_model_df.columns = ['revenue', 'months']
rank_model_df.head()

,revenue,months
pk_cid,,
16063,0,5
16203,40,4
16502,110,7
17457,1000,16
17590,0,10


In [101]:
rank_model_df.shape

(435782, 2)

In [102]:
EM_last_month = df.loc[df['pk_partition']=='2019-05-28',['pk_cid','em_acount']]
EM_last_month.set_index('pk_cid', inplace=True)

In [106]:
EM_last_month.shape

(440248, 1)

In [103]:
rank_model_df = pd.merge(rank_model_df, EM_last_month, how='left',left_index=True, right_index=True)

In [104]:
rank_model_df.head()

,revenue,months,em_acount
pk_cid,,,
16063,0,5,0
16203,40,4,1
16502,110,7,1
17457,1000,16,1
17590,0,10,0


In [105]:
rank_model_df.shape

(435782, 3)

In [108]:
# as said, age will be added as a one hot encoded feature so there is more information from where to draw patterns
df['age_cat'] = pd.cut(x=df['age'], bins=[0,18,30,40,50,60,70,80,110], labels=['u18','18-30','31-40','41-50','51-60','61-70','71-80','o80'])
df.head()

,pk_cid,pk_partition,country_id,region_code,gender,age,salary,short_term_deposit,loans,mortgage,...,entry_channel,active_customer,Total_products,year_partition,month_partition,year_month_partition,quarter_entry,year_month_entry,salary_LN,age_cat
0,1375586,2018-01-28,ES,Malaga,H,36,"87,218.10",0,0,0,...,others,1,1,2018,1,2018-01,2018-Q1,2018-01,11.38,31-40
1,1050611,2018-01-28,ES,Ciudad Real,V,24,"35,548.74",0,0,0,...,KHE,0,1,2018,1,2018-01,2015-Q3,2015-08,10.48,18-30
2,1050612,2018-01-28,ES,Ciudad Real,V,23,"122,179.11",0,0,0,...,KHE,0,1,2018,1,2018-01,2015-Q3,2015-08,11.71,18-30
3,1050613,2018-01-28,ES,Zaragoza,H,23,"119,775.54",1,0,0,...,KHD,0,1,2018,1,2018-01,2015-Q3,2015-08,11.69,18-30
4,1050614,2018-01-28,ES,Zaragoza,V,24,"89,257.08",0,0,0,...,KHE,1,1,2018,1,2018-01,2015-Q3,2015-08,11.40,18-30


In [109]:
_aux_age = df.loc[df['pk_partition']=='2019-05-28',['pk_cid','age_cat']]
_aux_age.set_index('pk_cid', inplace=True)
# filter out clients that are not appearing
rank_model_df = pd.merge(rank_model_df, _aux_age, how='left',left_index=True, right_index=True)
rank_model_df.head()

,revenue,months,em_acount,age_cat
pk_cid,,,,
16063,0,5,0,61-70
16203,40,4,1,61-70
16502,110,7,1,51-60
17457,1000,16,1,51-60
17590,0,10,0,51-60


In [111]:
# one hot encode the age categories
_dummy_df = pd.get_dummies(rank_model_df['age_cat'], prefix='age')
rank_model_df = pd.merge(rank_model_df, _dummy_df, how='left',left_index=True, right_index=True)
rank_model_df.drop('age_cat', axis=1, inplace=True)
rank_model_df.head()


,revenue,months,em_acount,age_u18,age_18-30,age_31-40,age_41-50,age_51-60,age_61-70,age_71-80,age_o80
pk_cid,,,,,,,,,,,
16063,0,5,0,0,0,0,0,0,1,0,0
16203,40,4,1,0,0,0,0,0,1,0,0
16502,110,7,1,0,0,0,0,1,0,0,0
17457,1000,16,1,0,0,0,0,1,0,0,0
17590,0,10,0,0,0,0,0,1,0,0,0


In [115]:
# make sure our 10k customers are not in the dataframe

index_10k = df_10k.index.tolist()
rank_model_df = rank_model_df[~rank_model_df.index.isin(index_10k)]

In [120]:
rank_model_df.to_csv('rank_model_df.csv')
df_10k.to_csv('df_10k.csv')

In [121]:
rank_model_df.rename(columns={"em_acount": "target_em"}, inplace=True)

In [122]:
rank_model_df.head()

,revenue,months,target_em,age_u18,age_18-30,age_31-40,age_41-50,age_51-60,age_61-70,age_71-80,age_o80
pk_cid,,,,,,,,,,,
16063,0,5,0,0,0,0,0,0,1,0,0
16203,40,4,1,0,0,0,0,0,1,0,0
16502,110,7,1,0,0,0,0,1,0,0,0
17457,1000,16,1,0,0,0,0,1,0,0,0
17799,160,16,1,0,0,0,1,0,0,0,0


In [125]:
rank_model_df['target_em'] = rank_model_df['target_em'].astype('uint8')

In [126]:
rank_model_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 425894 entries, 16063 to 1548207
Data columns (total 11 columns):
 #   Column     Non-Null Count   Dtype
---  ------     --------------   -----
 0   revenue    425894 non-null  int64
 1   months     425894 non-null  int64
 2   target_em  425894 non-null  uint8
 3   age_u18    425894 non-null  uint8
 4   age_18-30  425894 non-null  uint8
 5   age_31-40  425894 non-null  uint8
 6   age_41-50  425894 non-null  uint8
 7   age_51-60  425894 non-null  uint8
 8   age_61-70  425894 non-null  uint8
 9   age_71-80  425894 non-null  uint8
 10  age_o80    425894 non-null  uint8
dtypes: int64(2), uint8(9)
memory usage: 13.4 MB


In [ ]:
# machine learning model to compute probabilities

#### Split data

In [128]:
X_train_em, X_dev_em, y_train_em, y_dev_em = model_selection.train_test_split(
    rank_model_df.drop('target_em',axis=1),
    rank_model_df['target_em'],
    test_size=0.3,
    random_state=42
)

In [130]:
print(rank_model_df.shape)
print(X_train_em.shape)
print(X_dev_em.shape)

(425894, 11)
(298125, 10)
(127769, 10)


In [131]:
X_test_em, X_val_em, y_test_em, y_val_em = model_selection.train_test_split(
    X_dev_em,
    y_dev_em,
    test_size = 0.5,
    random_state=42
)

In [133]:
print(X_test_em.shape)
print(X_val_em.shape)

(63884, 10)
(63885, 10)


#### Model

In [134]:
split_dict_em = {
    "TRAINING": [X_train_em, y_train_em],
    "TESTING": [X_test_em, y_test_em]
}

In [141]:
# instantiate model
xgb_model_em = xgb.XGBClassifier(
    eta = 0.1,
    max_depth = 45,
    min_child_weight = 0.5,
    gamma = 5,
    random_state = 42,
    verbosity=0,
    use_label_encoder=False
)

# train 
xgb_model_em.fit(X = X_train_em, y = y_train_em)

c:\Users\javce\anaconda3\envs\nuclio_env\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              eta=0.1, gamma=5, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.100000001,
              max_delta_step=0, max_depth=45, min_child_weight=0.5, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=8,
              num_parallel_tree=1, predictor='auto', random_state=42,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', use_label_encoder=False,
              validate_parameters=1, verbosity=0)

In [142]:
print("################## em acount ##################")
print("\n____________ SCORES & EVALUATIONS ____________\n")
print("#################### RESULTS ###################")


for data in split_dict_em.items():
    pred = xgb_model_em.predict(data[1][0])
    confusion_matrix = metrics.confusion_matrix(data[1][1], pred)
    tn, fp, fn, tp = confusion_matrix.ravel()
    Accuracy = metrics.accuracy_score(data[1][1], pred)
    Precision = metrics.precision_score(data[1][1], pred)
    Recall = metrics.recall_score(data[1][1], pred)
    F_1_Score = metrics.f1_score(data[1][1], pred)


    probs = xgb_model_em.predict_proba(data[1][0])[:, 1]
    probs_mean = round(probs.mean()* 100, 2) 
    auc_score = roc_auc_score(data[1][1], probs)
    fpr, tpr, thresholds = roc_curve(data[1][1], probs)

    PPV, NPV = ((tp / (tp + fp)) * 100), ((tn / (fn + tn)) * 100)
    
    
    print(f"#################### {data[0]} ####################")
    print(f"Accuracy: {round(Accuracy, 5)} | Precision: {round(Precision, 5)} | Recall: {round(Recall, 5)} | F1_Score: {round(F_1_Score, 5)}")
    print(f"TN = {tn} | FN = {fn} | TP = {tp} | FP = {fp}")
    print(f"Positive prediction value: {round(PPV, 2)}% | Negative prediction value: {round(NPV, 2)}%")
    print("########## TOP FEATURES ##########")
    top_features = pd.Series(xgb_model_em.feature_importances_, index = data[1][0].columns).sort_values(ascending = False).head()
    print(top_features)
    print("\n")

################## em acount ##################

____________ SCORES & EVALUATIONS ____________

#################### RESULTS ###################
#################### TRAINING ####################
Accuracy: 0.9364 | Precision: 0.9417 | Recall: 0.96818 | F1_Score: 0.95475
TN = 79131 | FN = 6575 | TP = 200034 | FP = 12385
Positive prediction value: 94.17% | Negative prediction value: 92.33%
########## TOP FEATURES ##########
revenue     0.64
age_18-30   0.21
months      0.05
age_31-40   0.04
age_o80     0.01
dtype: float32


#################### TESTING ####################
Accuracy: 0.93239 | Precision: 0.93874 | Recall: 0.96486 | F1_Score: 0.95162
TN = 17089 | FN = 1547 | TP = 42476 | FP = 2772
Positive prediction value: 93.87% | Negative prediction value: 91.7%
########## TOP FEATURES ##########
revenue     0.64
age_18-30   0.21
months      0.05
age_31-40   0.04
age_o80     0.01
dtype: float32




In [ ]:
# Grid search

params_xgb = {
        'eta': [0.05, 0.1, 0.3],
        'min_child_weight': [0.5, 1, 5],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'max_depth': [5, 10, 30],
        'subsample': [0.8, 0.9, 1]
        }

grid_search_xgb = model_selection.GridSearchCV(
    estimator=xgb_model_em,
    param_grid=params_xgb,
    scoring='f1',
    cv=5,
    verbose=1
)

grid_search_xgb.fit(X_train_em, y_train_em)

print(grid_search_xgb.best_estimator_)

In [143]:
# validation data

y_val_pred_em = pd.DataFrame(xgb_model_em.predict(X_test_em), index = y_test_em.index, columns = ['pred'])
results_val_em = pd.DataFrame(y_test_em).join(y_val_pred_em, how = 'inner')

# metrics
Accuracy_val_em = metrics.accuracy_score(results_val_em['target_em'], results_val_em['pred'])
Precision_val_em = metrics.precision_score(results_val_em['target_em'], results_val_em['pred'])
Recall_val_em = metrics.recall_score(results_val_em['target_em'], results_val_em['pred'])
rf_f1_val_em = metrics.f1_score(y_test_em, y_val_pred_em)
print("Accuracy: ", Accuracy_val_em)
print("Precision: ", Precision_val_em)
print("Recall: ", Recall_val_em)
print("F1 score xgb: ",rf_f1_val_em)

Accuracy:  0.9323930874710412
Precision:  0.9387376237623762
Recall:  0.9648592781046271
F1 score xgb:  0.9516192268485847


Evaluation of our 10k chosen customers


In [147]:
# add the ages
df_10k = pd.merge(df_10k, _aux_age, how='left', left_index=True, right_index=True)
# one hot encode the ages
_dummy_df = pd.get_dummies(df_10k['age_cat'], prefix='age')
df_10k = pd.merge(df_10k, _dummy_df, how='left',left_index=True, right_index=True)
df_10k.drop('age_cat', axis=1, inplace=True)
df_10k.head()


,revenue,months,age_u18,age_18-30,age_31-40,age_41-50,age_51-60,age_61-70,age_71-80,age_o80
pk_cid,,,,,,,,,,
1045535,2430,16,0,0,1,0,0,0,0,0
1116106,2260,17,0,0,0,1,0,0,0,0
1020461,2260,17,0,0,1,0,0,0,0,0
1119050,2030,17,0,0,1,0,0,0,0,0
1209899,1950,17,0,0,1,0,0,0,0,0


In [153]:
proba_em = xgb_model_em.predict_proba(df_10k)
proba_em_df = pd.DataFrame(data = proba_em, index=df_10k.index, columns=['class1','class2'])
df_10k = pd.merge(df_10k, proba_em_df['class2'], how='left',left_index=True, right_index=True)
df_10k


,revenue,months,age_u18,age_18-30,age_31-40,age_41-50,age_51-60,age_61-70,age_71-80,age_o80,class2
pk_cid,,,,,,,,,,,
1045535,2430,16,0,0,1,0,0,0,0,0,0.13
1116106,2260,17,0,0,0,1,0,0,0,0,0.33
1020461,2260,17,0,0,1,0,0,0,0,0,0.22
1119050,2030,17,0,0,1,0,0,0,0,0,0.52
1209899,1950,17,0,0,1,0,0,0,0,0,0.26
...,...,...,...,...,...,...,...,...,...,...,...
1119048,50,0,0,1,0,0,0,0,0,0,0.86
1352778,50,0,0,0,1,0,0,0,0,0,0.62
1272462,50,0,0,1,0,0,0,0,0,0,0.86


In [154]:
df_10k[df_10k['class2']>0.5]

,revenue,months,age_u18,age_18-30,age_31-40,age_41-50,age_51-60,age_61-70,age_71-80,age_o80,class2
pk_cid,,,,,,,,,,,
1119050,2030,17,0,0,1,0,0,0,0,0,0.52
1185828,1180,17,0,0,0,1,0,0,0,0,0.51
1239232,1120,17,0,0,0,0,1,0,0,0,0.95
1260310,1120,17,0,0,1,0,0,0,0,0,0.96
1261703,1120,17,0,1,0,0,0,0,0,0,0.95
...,...,...,...,...,...,...,...,...,...,...,...
1119048,50,0,0,1,0,0,0,0,0,0,0.86
1352778,50,0,0,0,1,0,0,0,0,0,0.62
1272462,50,0,0,1,0,0,0,0,0,0,0.86


In [ ]:
df_10k[df_10k['class2']>0.5].index.to_list()

In [160]:
len(set(index_group1) & set(df_10k[df_10k['class2']>0.5].index.to_list()))

# Actually only 653 customers out of the 2769 seem to have more than random probabilities of purchasing the account

653

In [155]:
df_10k['class2'].describe()

count   10,000.00
mean         0.65
std          0.34
min          0.02
25%          0.41
50%          0.78
75%          0.96
max          1.00
Name: class2, dtype: float64

Interval of confidence - David's notebook

Calculate revenue range out of this + interval of confidence?

# **Alternative**

In [ ]:
# just to check something different, since we had such a low outcome from the previous model, let's see how does it go if I select 10k out of the people that have em account 
# to offer them debit card
# Same steps
# 1. Active
# 2. Non-active by revenue-months
# modelling

In [202]:
# manual selection of the new 10k
# keep only meaningful features
tgt_df = pd.concat([df[['pk_partition','pk_cid']],df[products],df['active_customer']],axis=1)
# filter out clients not in tgt_id (customers present in last month)
tgt_df = tgt_df[tgt_df['pk_cid'].isin(tgt_id)]

len(tgt_df['pk_cid'].unique())


440248

In [203]:
tgt_df = tgt_df*revenues_list

In [204]:
tgt_df.loc[:,'sum'] = tgt_df[products].sum(axis=1)
tgt_df.head()

,pk_partition,pk_cid,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,credit_card,payroll,pension_plan,payroll_account,emc_account,debit_card,em_acount,active_customer,sum
0,2018-01-28,1375586,0,0,0,0,0,0,0,0,0,0,0,0,10,1,10
1,2018-01-28,1050611,0,0,0,0,0,0,0,0,0,0,0,0,10,0,10
2,2018-01-28,1050612,0,0,0,0,0,0,0,0,0,0,0,0,10,0,10
3,2018-01-28,1050613,40,0,0,0,0,0,0,0,0,0,0,0,0,0,40
4,2018-01-28,1050614,0,0,0,0,0,0,0,0,0,0,0,0,10,1,10


In [205]:
rank_tgt = pd.DataFrame(index=tgt_id, columns=['revenue','months'])
rank_tgt = tgt_df.groupby('pk_cid').agg({'sum': 'sum', 'active_customer': 'sum'})
rank_tgt.columns = ['revenue', 'months']
rank_tgt.to_csv('rank_tgt.csv')
rank_tgt.head()

,revenue,months
pk_cid,,
16063,0,5
16203,50,5
16502,130,8
17457,1110,17
17590,0,11


In [206]:
rank_tgt.shape

(440248, 2)

In [207]:
products_per_customer = df.loc[df['pk_partition']=='2019-05-28'].apply(get_products, axis=1)    
products_per_customer = products_per_customer.apply(pd.Series)
products_per_customer.columns = ['products', 'id']
products_per_customer.set_index('id', inplace=True)
rank_tgt = pd.merge(rank_tgt, products_per_customer, how='left', left_index=True, right_index=True)
rank_tgt['products'].value_counts().head(10)

[em_acount]                                             255094
[]                                                      108751
[debit_card, em_acount]                                  22400
[emc_account, em_acount]                                  7825
[payroll, pension_plan, payroll_account, debit_card]      6009
[emc_account]                                             5745
[payroll_account]                                         4704
[payroll, pension_plan, payroll_account]                  4335
[payroll_account, debit_card]                             3109
[emc_account, debit_card, em_acount]                      2279
Name: products, dtype: int64

In [208]:
rank_tgt['products_str'] = rank_tgt['products'].astype('str')
rank_tgt['products_str'] = rank_tgt['products_str'].str.strip('[]').str.replace("'","")

In [209]:
# 255094 customers appearing in the last month w/o products
# from here we will have to make a selection of 10k

# First step:
# which ones of them have been active last month??

index_group1 = df.loc[(df['pk_partition']=='2019-05-28')&(df['active_customer']==1)&(df['em_acount']==1)&(df['Total_products']==1), 'pk_cid'].unique()
len(index_group1)

94195

In [210]:
# having so many, we can select the 10k out of this group directly, let's order by revenue and month
df_10k = rank_tgt[rank_tgt.index.isin(index_group1)]
# get the best 10k
df_10k = df_10k.sort_values(by=['revenue','months'], ascending=False).iloc[:10000]
# Remove the 10k from the df for training
dc_index = df_10k.index
rank_tgt = rank_tgt[~rank_tgt.index.isin(dc_index)]
print(len(rank_tgt))
print(len(df_10k))
print(len(rank_tgt) + len(df_10k))


430248
10000
440248


# **Chances**

In [211]:
# 1. Recompute revenue and months w/o taking into account last partition
# keep only meaningful features
tgt_model_df = pd.concat([df[['pk_partition','pk_cid']],df[products],df['active_customer']],axis=1)
# filter out clients not in tgt_id
tgt_model_df = tgt_model_df[tgt_model_df['pk_cid'].isin(tgt_id)]
# remove last month because it will used as a target in the modelling phase
tgt_model_df = tgt_model_df[tgt_model_df.pk_partition != '2019-05-28']

tgt_model_df = tgt_model_df*revenues_list
tgt_model_df.loc[:,'sum'] = tgt_model_df[products].sum(axis=1)
tgt_model_df.head()

,pk_partition,pk_cid,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,credit_card,payroll,pension_plan,payroll_account,emc_account,debit_card,em_acount,active_customer,sum
0,2018-01-28,1375586,0,0,0,0,0,0,0,0,0,0,0,0,10,1,10
1,2018-01-28,1050611,0,0,0,0,0,0,0,0,0,0,0,0,10,0,10
2,2018-01-28,1050612,0,0,0,0,0,0,0,0,0,0,0,0,10,0,10
3,2018-01-28,1050613,40,0,0,0,0,0,0,0,0,0,0,0,0,0,40
4,2018-01-28,1050614,0,0,0,0,0,0,0,0,0,0,0,0,10,1,10


In [212]:
rank_model_df = tgt_model_df.groupby('pk_cid').agg({'sum': 'sum', 'active_customer': 'sum'})
rank_model_df.columns = ['revenue', 'months']
rank_model_df.head()

,revenue,months
pk_cid,,
16063,0,5
16203,40,4
16502,110,7
17457,1000,16
17590,0,10


In [213]:
DC_last_month = df.loc[df['pk_partition']=='2019-05-28',['pk_cid','debit_card']]
DC_last_month.set_index('pk_cid', inplace=True)

In [214]:
rank_model_df = pd.merge(rank_model_df, DC_last_month, how='left',left_index=True, right_index=True)
rank_model_df.head()

,revenue,months,debit_card
pk_cid,,,
16063,0,5,0
16203,40,4,0
16502,110,7,0
17457,1000,16,1
17590,0,10,0


In [215]:
# as said, age will be added as a one hot encoded feature so there is more information from where to draw patterns

rank_model_df = pd.merge(rank_model_df, _aux_age, how='left',left_index=True, right_index=True)
# one hot encode the age categories
_dummy_df = pd.get_dummies(rank_model_df['age_cat'], prefix='age')
rank_model_df = pd.merge(rank_model_df, _dummy_df, how='left',left_index=True, right_index=True)
rank_model_df.drop('age_cat', axis=1, inplace=True)
rank_model_df.head()

,revenue,months,debit_card,age_u18,age_18-30,age_31-40,age_41-50,age_51-60,age_61-70,age_71-80,age_o80
pk_cid,,,,,,,,,,,
16063,0,5,0,0,0,0,0,0,1,0,0
16203,40,4,0,0,0,0,0,0,1,0,0
16502,110,7,0,0,0,0,0,1,0,0,0
17457,1000,16,1,0,0,0,0,1,0,0,0
17590,0,10,0,0,0,0,0,1,0,0,0


In [216]:
# make sure our 10k customers are not in the dataframe
index_10k = df_10k.index.tolist()
rank_model_df = rank_model_df[~rank_model_df.index.isin(index_10k)]

In [218]:
rank_model_df.rename(columns={"debit_card": "target_dc"}, inplace=True)
rank_model_df['target_em'] = rank_model_df['target_dc'].astype('uint8')
rank_model_df.head()

,revenue,months,target_dc,age_u18,age_18-30,age_31-40,age_41-50,age_51-60,age_61-70,age_71-80,age_o80,target_em
pk_cid,,,,,,,,,,,,
16063,0,5,0,0,0,0,0,0,1,0,0,0
16203,40,4,0,0,0,0,0,0,1,0,0,0
16502,110,7,0,0,0,0,0,1,0,0,0,0
17457,1000,16,1,0,0,0,0,1,0,0,0,1
17590,0,10,0,0,0,0,0,1,0,0,0,0


Split data

In [219]:
X_train_dc, X_dev_dc, y_train_dc, y_dev_dc = model_selection.train_test_split(
    rank_model_df.drop('target_dc',axis=1),
    rank_model_df['target_dc'],
    test_size=0.3,
    random_state=42
)

In [220]:
print(rank_model_df.shape)
print(X_train_dc.shape)
print(X_dev_dc.shape)

(425782, 12)
(298047, 11)
(127735, 11)


In [221]:
X_test_dc, X_val_dc, y_test_dc, y_val_dc = model_selection.train_test_split(
    X_dev_dc,
    y_dev_dc,
    test_size = 0.5,
    random_state=42
)

In [222]:
print(X_test_dc.shape)
print(X_val_dc.shape)

(63867, 11)
(63868, 11)


Model

In [223]:
split_dict_dc = {
    "TRAINING": [X_train_dc, y_train_dc],
    "TESTING": [X_test_dc, y_test_dc]
}

In [224]:
# instantiate model
xgb_model_dc = xgb.XGBClassifier(
    eta = 0.1,
    max_depth = 45,
    min_child_weight = 0.5,
    gamma = 5,
    random_state = 42,
    verbosity=0,
    use_label_encoder=False
)

# train 
xgb_model_dc.fit(X = X_train_dc, y = y_train_dc)

c:\Users\javce\anaconda3\envs\nuclio_env\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              eta=0.1, gamma=5, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.100000001,
              max_delta_step=0, max_depth=45, min_child_weight=0.5, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=8,
              num_parallel_tree=1, predictor='auto', random_state=42,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', use_label_encoder=False,
              validate_parameters=1, verbosity=0)

**Limpiar el predict proba aqui**

In [225]:
print("################## em acount ##################")
print("\n____________ SCORES & EVALUATIONS ____________\n")
print("#################### RESULTS ###################")


for data in split_dict_dc.items():
    pred = xgb_model_dc.predict(data[1][0])
    confusion_matrix = metrics.confusion_matrix(data[1][1], pred)
    tn, fp, fn, tp = confusion_matrix.ravel()
    Accuracy = metrics.accuracy_score(data[1][1], pred)
    Precision = metrics.precision_score(data[1][1], pred)
    Recall = metrics.recall_score(data[1][1], pred)
    F_1_Score = metrics.f1_score(data[1][1], pred)


    probs = xgb_model_dc.predict_proba(data[1][0])[:, 1]
    probs_mean = round(probs.mean()* 100, 2) 
    auc_score = roc_auc_score(data[1][1], probs)
    fpr, tpr, thresholds = roc_curve(data[1][1], probs)

    PPV, NPV = ((tp / (tp + fp)) * 100), ((tn / (fn + tn)) * 100)
    
    
    print(f"#################### {data[0]} ####################")
    print(f"Accuracy: {round(Accuracy, 5)} | Precision: {round(Precision, 5)} | Recall: {round(Recall, 5)} | F1_Score: {round(F_1_Score, 5)}")
    print(f"TN = {tn} | FN = {fn} | TP = {tp} | FP = {fp}")
    print(f"Positive prediction value: {round(PPV, 2)}% | Negative prediction value: {round(NPV, 2)}%")
    print("########## TOP FEATURES ##########")
    top_features = pd.Series(xgb_model_dc.feature_importances_, index = data[1][0].columns).sort_values(ascending = False).head()
    print(top_features)
    print("\n")

################## em acount ##################

____________ SCORES & EVALUATIONS ____________

#################### RESULTS ###################
#################### TRAINING ####################
Accuracy: 1.0 | Precision: 1.0 | Recall: 1.0 | F1_Score: 1.0
TN = 267970 | FN = 0 | TP = 30077 | FP = 0
Positive prediction value: 100.0% | Negative prediction value: 100.0%
########## TOP FEATURES ##########
target_em   1.00
revenue     0.00
months      0.00
age_u18     0.00
age_18-30   0.00
dtype: float32


#################### TESTING ####################
Accuracy: 1.0 | Precision: 1.0 | Recall: 1.0 | F1_Score: 1.0
TN = 57348 | FN = 0 | TP = 6519 | FP = 0
Positive prediction value: 100.0% | Negative prediction value: 100.0%
########## TOP FEATURES ##########
target_em   1.00
revenue     0.00
months      0.00
age_u18     0.00
age_18-30   0.00
dtype: float32




In [226]:
# validation data

y_val_pred_dc = pd.DataFrame(xgb_model_dc.predict(X_test_dc), index = y_test_dc.index, columns = ['pred'])
results_val_dc = pd.DataFrame(y_test_dc).join(y_val_pred_dc, how = 'inner')

# metrics
Accuracy_val_dc = metrics.accuracy_score(results_val_dc['target_dc'], results_val_dc['pred'])
Precision_val_dc = metrics.precision_score(results_val_dc['target_dc'], results_val_dc['pred'])
Recall_val_dc = metrics.recall_score(results_val_dc['target_dc'], results_val_dc['pred'])
rf_f1_val_dc= metrics.f1_score(y_test_dc, y_val_pred_dc)
print("Accuracy: ", Accuracy_val_dc)
print("Precision: ", Precision_val_dc)
print("Recall: ", Recall_val_dc)
print("F1 score xgb: ",rf_f1_val_dc)

Accuracy:  1.0
Precision:  1.0
Recall:  1.0
F1 score xgb:  1.0


Evaluation of our 10k chosen customers


In [ ]:
# add the ages
df_10k = pd.merge(df_10k, _aux_age, how='left', left_index=True, right_index=True)
# one hot encode the ages
_dummy_df = pd.get_dummies(df_10k['age_cat'], prefix='age')
df_10k = pd.merge(df_10k, _dummy_df, how='left',left_index=True, right_index=True)
df_10k.drop('age_cat', axis=1, inplace=True)
df_10k.head()


In [ ]:
proba_dc = xgb_model_em.predict_proba(df_10k)
proba_dc_df = pd.DataFrame(data = proba_dc, index=df_10k.index, columns=['class1','class2'])
df_10k = pd.merge(df_10k, proba_dc_df['class2'], how='left',left_index=True, right_index=True)
df_10k

In [ ]:
df_10k[df_10k['class2']>0.5]

In [ ]:
len(set(index_group1) & set(df_10k[df_10k['class2']>0.5].index.to_list()))


In [ ]:
df_10k['class2'].describe()

Interval of confidence - David's notebook

Calculate revenue range out of this + interval of confidence?

# **limitations and future steps**

In [ ]:
# another option, do not prefilter and use all these customers as "new data", select the 10k with best rate response